In [1]:
# 导入必要的包
import cv2
import numpy as np
import xml.etree.cElementTree as ET

In [2]:
# 用于训练的数量
num_of_pic = 9

In [3]:
# 读取xml文件并解析
et = ET.parse('out_camera_data.xml')
# 得到根节点
element = et.getroot()

# 找到外参数，并储存在extrinsic_parameters中
extrinsic_parameters = element.find('extrinsic_parameters')
# 取出其中的data，转变成为list类型，并变成对应的形状
ext = list(extrinsic_parameters.find('data').text.split())
ext = list(map(float, ext))
ext = np.array(ext).reshape(num_of_pic, 6)

# 找到相机矩阵，并储存在camera_matrix中
camera_matrix = element.find('camera_matrix')
cam = list(camera_matrix.find('data').text.split())
cam = list(map(float, cam))
cam = np.array(cam).reshape(3, 3)

# 找到最后的点位置，并储存在image_points中
image_points = element.find('image_points')
pts = list(image_points.find('data').text.split())
pts = list(map(float, pts))
pts = np.array(pts).reshape(num_of_pic, 64, 2)

grid_points = element.find('grid_points')
pts_world = list(grid_points.text.split())
pts_world = list(map(float, pts_world))
pts_world = np.array(pts_world).reshape(64, 3)

In [4]:
# 选择其中一组点（总共64个）进行恢复
pts = pts[0, :, :]
pts = np.hstack((pts, np.ones((64, 1))))
print(pts[0])

[167.955002 249.612503   1.      ]


In [5]:
# position中有的是平移向量
position = ext[:, 3:]
# rotation中有的是旋转向量
rotation = ext[:, :3]

In [6]:
# 这一步尝试绘制一张图像，验证一下给出点的坐标是以图片左上角为原点的坐标
filename = './images/1.jpg'
img = cv2.imread(filename)

point_size = 1
point_color = (0, 0, 255)
thickness = 4

for pt in pts:
    cv2.circle(img, (int(pt[0]), int(pt[1])), point_size, point_color, thickness)

cv2.imwrite('1.png', img)

True

In [7]:
# 重建得到相机的位置
result = []

for i in range(rotation.shape[0]):
    rot = cv2.Rodrigues(rotation[i])
    R = rot[0]
    # 参见公式
    result.append(np.dot(np.linalg.inv(R), -position[i].T))

In [8]:
# 重建第一组点
rot = cv2.Rodrigues(rotation[0])
R = rot[0]
# [R T]，3 x 4
mat = np.hstack((R, position[0].reshape((3, 1))))
print(mat.shape)

(3, 4)


In [9]:
# 计算得到64个点重建的三维坐标
tmp1 = np.dot(np.linalg.inv(cam), pts.T)
out = np.dot(np.linalg.pinv(mat), tmp1)

In [11]:
# 齐次坐标，最后一位应该为1，因此除以做齐次化
out = (out / out[3, :]).T

In [12]:
# 需要安装，pip install plyfile
# 创建ply文件
from plyfile import PlyData, PlyElement

def write_ply(save_path, points, pts, text=True):
    points = [(points[i, 0], points[i, 1], points[i, 2]) for i in range(points.shape[0])]

    for i in range(pts.shape[0]):
        points.append((pts[i, 0], pts[i, 1], pts[i, 2]))

    vertex = np.array(points, dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4')])
    el = PlyElement.describe(vertex, 'vertex', comments=['vertices'])
    PlyData([el], text=text).write(save_path)

write_ply('new.ply', np.array(result), pts_world, True)

In [14]:
origin = [[100, 100,  0], [100, 150,  0], [150, 150,  0], [150, 100,  0],\
    [100, 100, -50], [100, 150, -50], [150, 150, -50], [150, 100, -50]]

origin = np.array(origin)
origin = np.hstack((origin, np.ones(8).reshape(8, 1)))

In [ ]:
import open3d as o3d
import numpy as np

model_path = 'cat_mod_1.ply'
model = o3d.io.read_point_cloud(model_path)
pts = o3d.np.asarray(model.points)

pts = origin + np.array([200, 200, 50])

In [16]:
tmp1 = np.dot(mat, origin.T)
out = np.dot(cam, tmp1)
out = out / out[2, :]

In [20]:
# 这一步尝试绘制一张图像，验证一下给出点的坐标是以图片左上角为原点的坐标
filename = './images/1.jpg'
img = cv2.imread(filename)

def draw_lines(img, points):
    pairs = []
    pairs += [(i, i+4) for i in range(4)]
    pairs += [(i, (i+1)%4) for i in range(4)]
    pairs += [(i+4, (i+1)%4 + 4) for i in range(4)]
    for tup in pairs:
        pt1 = (int(points[tup[0], 0]), int(points[tup[0], 1]))
        pt2 = (int(points[tup[1], 0]), int(points[tup[1], 1]))
        cv2.line(img, pt1, pt2, (0, 0, 255), 4, 3)

    cv2.imwrite('1.png', img)

print(out.shape)
draw_lines(img, out.T)

(3, 8)


In [21]:
filename = './images/1.jpg'
img = cv2.imread(filename)

point_size = 1
point_color = (0, 0, 255)
thickness = 4

for pt in out.T:
    cv2.circle(img, (int(pt[0]), int(pt[1])), point_size, point_color, thickness)

cv2.imwrite('1.png', img)

True

In [22]:
for i in range(num_of_pic):
    # 重建第一组点
    rot = cv2.Rodrigues(rotation[i])
    R = rot[0]
    # [R T]，3 x 4
    mat = np.hstack((R, position[i].reshape((3, 1))))
    # print(mat.shape)
    tmp1 = np.dot(mat, origin.T)
    out = np.dot(cam, tmp1)
    out = out / out[2, :]

    filename = './images/' + str(i+1) + '.jpg'
    img = cv2.imread(filename)
    draw_lines(img, out.T)

    cv2.imwrite(str(i+1)+'.png', img)